In [35]:
# !pip install -r ../exp_requirements.txt

In [36]:
# !pip uninstall virny -y

In [37]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@development

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@development

In [9]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

# Folktables GA Dataset With Random Nulls

## Import dependencies

In [11]:
import os
import copy

from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from virny.utils.custom_initializers import create_config_obj, create_models_metrics_dct_from_database_df
from virny.datasets.data_loaders import ACSIncomeDataset
from source.user_interfaces.experiment_interface import run_exp_iter_with_models_stress_testing
from source.error_injectors.random_nulls_injector_v2 import RandomNullsInjectorV2
from source.utils.custom_initializers import create_experiment_data_loader
from source.utils.db_functions import read_model_metric_dfs_from_db
from source.preprocessing.basic_preprocessing import get_null_imputer_preprocessor

from configs.constants import NUM_METRICS_COMPUTATION_RUNS, EXPERIMENT_SEEDS, TEST_SET_FRACTION

## Initialize input variables for the experiment

In [12]:
ROOT_DIR = os.path.join(os.getcwd(), "..", "..")
EXPERIMENT_NAME = 'stress_testing_nulls'
DB_COLLECTION_NAME = f'{EXPERIMENT_NAME}_results'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)
COLUMNS_TO_TRANSFORM = ['SCHL', 'COW', 'MAR', 'OCCP', 'POBP', 'RELP', 'WKHP']
INJECTOR_CONFIG_LST = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
MAX_NUM_COLUMNS_TO_EFFECT = 3

In [13]:
data_loader = ACSIncomeDataset(state=['GA'], year=2018, with_nulls=False)
data_loader.X_data.head()

,SCHL,COW,MAR,OCCP,POBP,RELP,SEX,RAC1P,AGEP,WKHP
2,20,1,5,4760,13,17,2,2,23,20.0
4,19,1,5,4760,13,16,2,1,20,20.0
6,19,1,5,4252,2,17,1,1,27,50.0
7,13,1,5,7200,13,16,1,2,23,32.0
10,21,2,5,2205,45,17,2,1,23,15.0


In [14]:
data_loader.full_df.shape

(50915, 10)

In [15]:
data_loader.full_df.isna().sum()

SCHL     0
COW      0
MAR      0
OCCP     0
POBP     0
RELP     0
SEX      0
RAC1P    0
AGEP     0
WKHP     0
dtype: int64

### Define a db writer and custom fields to insert into your database

In [16]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [17]:
custom_table_fields_dct = {
    'error_type': 'Random Nulls',
    'error_percentages': '0%,5%,10%,20%,30%,40%,50%',
    # 'error_percentages': '0%,10%,20%,40%,60%,80%,100%',
    # 'preprocessing_technique': 'cat: mode, num: mean',
    'preprocessing_technique': 'cat: mode, num: median',
}

In [19]:
import uuid

# custom_table_fields_dct['session_uuid'] = 'c53d250b-5ba9-4d91-a444-ed7eb7919de5'
# custom_table_fields_dct['session_uuid'] = 'e38b6e20-e4e7-4791-89d3-5e371513d2ea'
# custom_table_fields_dct['session_uuid'] = '2f6541f0-2a77-446a-a5be-92d7b4187052'
# custom_table_fields_dct['session_uuid'] = 'a4febf95-81bf-4aee-9b51-a690d596d5d2'

# custom_table_fields_dct['session_uuid'] = '6feaa1ae-df84-4ff3-8231-94945531e397'
custom_table_fields_dct['session_uuid'] = '42191a73-175c-465e-839c-635d9f83d9e5'
# custom_table_fields_dct['session_uuid'] = str(uuid.uuid4())
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  42191a73-175c-465e-839c-635d9f83d9e5


### Create a metrics computation config object

In [20]:
config_yaml_path = 'experiment_config.yaml'
age_range = [i for i in range(30, 51)]
config_yaml_content = \
f"""
dataset_name: Folktables_Income_GA_2018
bootstrap_fraction: 0.8
# n_estimators: 50
n_estimators: 10
# num_runs: {NUM_METRICS_COMPUTATION_RUNS}
num_runs: 2
sensitive_attributes_dct: {{'SEX': '1', 'RAC1P': '1', 'AGEP': {age_range}, 'SEX & RAC1P & AGEP': None}}
"""

with open(config_yaml_path, 'w', encoding='utf-8') as f:
    f.write(config_yaml_content)

In [21]:
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

### Define tuning parameter for models

In [22]:
def get_models_params_for_tuning(models_tuning_seed):
    return {
        'DecisionTreeClassifier': {
            'model': DecisionTreeClassifier(random_state=models_tuning_seed),
            'params': {
                "max_depth": [20, 30],
                "min_samples_split" : [0.1],
                "max_features": ['sqrt'],
                "criterion": ["gini", "entropy"]
            }
        },
        'LogisticRegression': {
            'model': LogisticRegression(random_state=models_tuning_seed),
            'params': {
                'penalty': ['l2'],
                'C' : [0.0001, 0.1, 1, 100],
                'solver': ['newton-cg', 'lbfgs'],
                'max_iter': [250],
            }
        },
        # 'RandomForestClassifier': {
        #     'model': RandomForestClassifier(random_state=models_tuning_seed),
        #     'params': {
        #         "max_depth": [6, 10],
        #         "min_samples_leaf": [1],
        #         "n_estimators": [50, 100],
        #         "max_features": [0.6]
        #     }
        # },
        # 'XGBClassifier': {
        #     'model': XGBClassifier(random_state=models_tuning_seed, verbosity=0),
        #     'params': {
        #         'learning_rate': [0.1],
        #         'n_estimators': [200],
        #         'max_depth': [5, 7],
        #         'lambda':  [10, 100]
        #     }
        # },
        # 'KNeighborsClassifier': {
        #     'model': KNeighborsClassifier(),
        #     'params': {
        #         'n_neighbors' : [5, 7, 9, 11, 13, 15, 25],
        #         'weights' : ['uniform', 'distance'],
        #         'metric' : ['minkowski', 'euclidean', 'manhattan']
        #     }
        # },
        # 'MLPClassifier': {
        #     'model': MLPClassifier(random_state=models_tuning_seed),
        #     'params': {
        #         'hidden_layer_sizes':[(100,), (100,100,), (100,50,100,)],
        #         'activation': ['logistic', 'tanh', 'relu'],
        #         'solver': ['lbfgs', 'sgd', 'adam'],
        #         'learning_rate': ['constant', 'invscaling', 'adaptive']
        #     }
        # }
    }

## Run experiments

### Experiment iteration 1

In [23]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

error_injector = RandomNullsInjectorV2(experiment_seed, columns_to_transform=COLUMNS_TO_TRANSFORM, row_idx_nulls_percentage=0.0,
                                       max_num_columns_to_effect=MAX_NUM_COLUMNS_TO_EFFECT)
models_params_for_tuning = get_models_params_for_tuning(experiment_seed)
# Create a transformed data loader
exp_iter_data_loader = create_experiment_data_loader(data_loader)
exp_iter_data_loader.columns_with_nulls = COLUMNS_TO_TRANSFORM
# preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader, categorical_trimmed=0.3)
preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader)

In [24]:
run_exp_iter_with_models_stress_testing(data_loader=exp_iter_data_loader,
                                        experiment_seed=experiment_seed,
                                        test_set_fraction=TEST_SET_FRACTION,
                                        db_writer_func=db_writer_func,
                                        error_injector=error_injector,
                                        injector_config_lst=INJECTOR_CONFIG_LST,
                                        preprocessor=preprocessor,
                                        models_params_for_tuning=models_params_for_tuning,
                                        metrics_computation_config=metrics_computation_config,
                                        custom_table_fields_dct=custom_table_fields_dct,
                                        with_tuning=True,
                                        save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                        tuned_params_df_path=None,
                                        verbose=True)

2023-04-25 14:39:54 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'error_percentages': '0%,5%,10%,20%,30%,40%,50%',
 'error_type': 'Random Nulls',
 'experiment_iteration': 'Exp_iter_1',
 'injector_config_lst': '[0.05, 0.1, 0.2, 0.3, 0.4, 0.5]',
 'model_init_seed': 100,
 'preprocessing_technique': 'cat: mode, num: median',
 'session_uuid': '42191a73-175c-465e-839c-635d9f83d9e5'}




2023-04-25 14:39:56 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([85878, 49139, 95525, 25749, 83799, 18626, 85209, 63885, 61619,
            85769, 94822, 70913, 62567, 45304, 39090, 21650, 79431, 95575,
            15027, 57309],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([85878, 49139, 95525, 25749, 83799, 18626, 85209, 63885, 61619,
            85769, 94822, 70913, 62567, 45304, 39090, 21650, 79431, 95575,
            15027, 57309],
           dtype='int64')
error_injector.seed --  101
transformed_X_test:
 SCHL     90
COW      63
MAR      70
OCCP     77
POBP     62
RELP     65
SEX       0
RAC1P     0
AGEP      0
WKHP     79
dtype: int64
error_injector.seed --  102
transformed_X_test:
 SCHL     166
COW      140
MAR      138
OCCP     153
POBP     148
RELP     151
SEX        0
RAC1P      0
AGEP       0
WKHP     149
dtype: int64
error_injector.seed --  103
transformed_X_test:
 SCHL     311
COW      267
MAR      276
OCCP     302
POBP     279
RELP  

2023-04-25 14:45:20 experiment_interface.py INFO    : Models are tuned and saved to a file


...

[CV 2/3; 1/4] START criterion=gini, max_depth=20, max_features=sqrt, min_samples_split=0.1
[CV 2/3; 1/4] END criterion=gini, max_depth=20, max_features=sqrt, min_samples_split=0.1; Accuracy_Score: (test=0.726) F1_Score: (test=0.671) total time=   4.1s
[CV 2/3; 1/8] START C=0.0001, max_iter=250, penalty=l2, solver=newton-cg........
[CV 2/3; 1/8] END C=0.0001, max_iter=250, penalty=l2, solver=newton-cg; Accuracy_Score: (test=0.692) F1_Score: (test=0.548) total time=  25.6s
[CV 2/3; 5/8] START C=1, max_iter=250, penalty=l2, solver=newton-cg.............
[CV 2/3; 5/8] END C=1, max_iter=250, penalty=l2, solver=newton-cg; Accuracy_Score: (test=0.821) F1_Score: (test=0.802) total time= 1.4min
[CV 1/3; 2/4] START criterion=gini, max_depth=30, max_features=sqrt, min_samples_split=0.1
[CV 1/3; 2/4] END criterion=gini, max_depth=30, max_features=sqrt, min_samples_split=0.1; Accuracy_Score: (test=0.742) F1_Score: (test=0.689) total time=   4.2s
[CV 1/3; 2/8] START C=0.0001, max_iter=250, penalty=l

Analyze models in one run:   0%|          | 0/2 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

[CV 3/3; 2/4] START criterion=gini, max_depth=30, max_features=sqrt, min_samples_split=0.1
[CV 3/3; 2/4] END criterion=gini, max_depth=30, max_features=sqrt, min_samples_split=0.1; Accuracy_Score: (test=0.740) F1_Score: (test=0.681) total time=   3.7s
[CV 2/3; 4/4] START criterion=entropy, max_depth=30, max_features=sqrt, min_samples_split=0.1
[CV 2/3; 4/4] END criterion=entropy, max_depth=30, max_features=sqrt, min_samples_split=0.1; Accuracy_Score: (test=0.710) F1_Score: (test=0.673) total time=   1.8s
[CV 1/3; 3/8] START C=0.1, max_iter=250, penalty=l2, solver=newton-cg...........
[CV 1/3; 3/8] END C=0.1, max_iter=250, penalty=l2, solver=newton-cg; Accuracy_Score: (test=0.811) F1_Score: (test=0.790) total time= 1.2min
[CV 3/3; 7/8] START C=100, max_iter=250, penalty=l2, solver=newton-cg...........
[CV 3/3; 7/8] END C=100, max_iter=250, penalty=l2, solver=newton-cg; Accuracy_Score: (test=0.812) F1_Score: (test=0.792) total time= 3.1min
[CV 1/3; 3/4] START criterion=entropy, max_depth

### Experiment iteration 2

In [131]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

error_injector = RandomNullsInjectorV2(experiment_seed, columns_to_transform=COLUMNS_TO_TRANSFORM, row_idx_nulls_percentage=0.0,
                                       max_num_columns_to_effect=MAX_NUM_COLUMNS_TO_EFFECT)
models_params_for_tuning = get_models_params_for_tuning(experiment_seed)
# Create a transformed data loader
exp_iter_data_loader = create_experiment_data_loader(data_loader)
exp_iter_data_loader.columns_with_nulls = COLUMNS_TO_TRANSFORM
# preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader, categorical_trimmed=0.3)
preprocessor = get_null_imputer_preprocessor(exp_iter_data_loader)

In [132]:
run_exp_iter_with_models_stress_testing(data_loader=exp_iter_data_loader,
                                        experiment_seed=experiment_seed,
                                        test_set_fraction=TEST_SET_FRACTION,
                                        db_writer_func=db_writer_func,
                                        error_injector=error_injector,
                                        injector_config_lst=INJECTOR_CONFIG_LST,
                                        preprocessor=preprocessor,
                                        models_params_for_tuning=models_params_for_tuning,
                                        metrics_computation_config=metrics_computation_config,
                                        custom_table_fields_dct=custom_table_fields_dct,
                                        with_tuning=True,
                                        save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                        tuned_params_df_path=None,
                                        verbose=True)

2023-04-23 23:17:11 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'error_percentages': '0%,10%,20%,40%,60%,80%,100%',
 'error_type': 'Random Nulls',
 'experiment_iteration': 'Exp_iter_2',
 'injector_config_lst': '[0.1, 0.2, 0.4, 0.6, 0.8, 1.0]',
 'model_init_seed': 200,
 'preprocessing_technique': 'cat: mode_trimmed_0.3, num: median',
 'session_uuid': 'a4febf95-81bf-4aee-9b51-a690d596d5d2'}


[CV 2/3; 1/4] START criterion=gini, max_depth=20, max_features=sqrt, min_samples_split=0.1
[CV 2/3; 1/4] END criterion=gini, max_depth=20, max_features=sqrt, min_samples_split=0.1; Accuracy_Score: (test=0.747) F1_Score: (test=0.727) total time=   0.1s
[CV 1/3; 4/4] START criterion=entropy, max_depth=30, max_features=sqrt, min_samples_split=0.1
[CV 1/3; 4/4] END criterion=entropy, max_depth=30, max_features=sqrt, min_samples_split=0.1; Accuracy_Score: (test=0.745) F1_Score: (test=0.729) total time=   0.1s
[CV 3/3; 4/4] START criterion=entropy, max_depth=30, max_features=sqrt, min_samples_split=0.1
[CV 3/3; 4/4] END criterion=entropy, 

2023-04-23 23:17:13 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([20518, 20570, 38530, 39945,  1713, 19318, 15625, 36291, 26020,
            18954,  7039, 40519, 42300,  5620, 39590, 39048, 33565, 18881,
            43190,  5750],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([20518, 20570, 38530, 39945,  1713, 19318, 15625, 36291, 26020,
            18954,  7039, 40519, 42300,  5620, 39590, 39048, 33565, 18881,
            43190,  5750],
           dtype='int64')
transformed_X_test.isnull().sum() --  MAR         42
MIL         55
ESP         48
MIG         51
DREM        49
NATIVITY    53
DIS         65
DEAR        70
DEYE        62
SEX          0
RAC1P        0
RELP        62
CIT         69
ANC         65
SCHL        52
AGEP         0
dtype: int64
transformed_X_test.isnull().sum() --  MAR         142
MIL         156
ESP         141
MIG         149
DREM        152
NATIVITY    150
DIS         150
DEAR        148
DEYE        146
SEX           0
RAC1P

2023-04-23 23:19:23 experiment_interface.py INFO    : Models are tuned and saved to a file


...

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/2 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/10 [00:00<?, ?it/s]

In [54]:
client.close()